<a href="https://colab.research.google.com/github/mezlet/PPI-Inhibitors-main/blob/main/PPI_Inhibitor_COMPLETE_END_TO_END.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complete PPI Inhibitor Prediction - Single Notebook
## ALL Components Included

**Paper:** Yaseen et al. "Predicting small-molecule inhibition of protein complexes"

**GitHub:** https://github.com/adibayaseen/PPI-Inhibitors

---

### ✅ This notebook includes EVERYTHING:
- All corrected functions (atom1, res1, neigh1)
- Complete GNN architecture  
- Dense class
- BalancedDataset & create_balanced_loader
- BinaryBalancedSampler
- readFile & processProtein utilities
- Full LOCO cross-validation
- Visualization

### 🎯 Expected Results:
- **Mean AUC-ROC:** 0.86 ± 0.09
- **Mean AUC-PR:** 0.39 ± 0.24

### ⏱️ Runtime:
- Setup: 5-10 minutes
- Full training (23 complexes): 12-24 hours on GPU

In [3]:
# Environment check
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Running in Google Colab")
    print("⚠️  Make sure Runtime -> Change runtime type -> GPU is enabled")
else:
    print("✓ Running locally")

import warnings
warnings.filterwarnings('ignore')

✓ Running in Google Colab
⚠️  Make sure Runtime -> Change runtime type -> GPU is enabled


In [2]:
# Install dependencies
# !pip install -q --upgrade pip setuptools wheel
# !pip install -q rdkit biopython==1.81 torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 \
#               torch-geometric==2.5.3 tqdm==4.66.2 pandas==2.1.4 numpy==1.26.4 \
#               scikit-learn==1.3.2 matplotlib==3.8.3 seaborn==0.13.2 networkx==3.2.1 gdown
!pip install --upgrade pip setuptools wheel
!pip install rdkit biopython==1.81 torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 torch-geometric==2.5.3 tqdm==4.66.2 pandas==2.1.4 numpy==1.26.4 scikit-learn==1.3.2 matplotlib==3.8.3 seaborn==0.13.2 networkx==3.2.1 gdown

print("\n✓ Packages installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 35.4 MB/s  0:00:11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 117.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 87.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 92.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 23.2 MB/s  0:00:11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 186.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 200.1 MB/s  0:00:00
   ━━━━━━━━━━━


✓ Packages installed


In [1]:
# Clone repository and download data
!rm -rf PPI-Inhibitors
!git clone https://github.com/adibayaseen/PPI-Inhibitors
%cd PPI-Inhibitors

# Download datasets
!mkdir -p Data
%cd Data
!wget -q https://github.com/adibayaseen/PPI-Inhibitors/raw/01ad4975fb9133825b1bf9e71b64fcdaaa5e4d8b/Data/2p2iComplexPairs.txt
!wget -q https://github.com/adibayaseen/PPI-Inhibitors/raw/01ad4975fb9133825b1bf9e71b64fcdaaa5e4d8b/Data/2p2iInhibitorsSMILES.txt
!wget -q https://github.com/adibayaseen/PPI-Inhibitors/raw/2d6bd03422602ec19147870c487e64018b52660f/Data/WriteAllexamplesRandomBindersIdsAll_24JAN_Binary.txt
!wget -q https://github.com/adibayaseen/PPI-Inhibitors/raw/b1e45884f61f792399abad2e4492f48083ab1093/Data/BindersWithComplexname.csv
%cd ..

print("\n✓ Repository cloned and datasets downloaded")

Cloning into 'PPI-Inhibitors'...
remote: Enumerating objects: 1341, done.
remote: Counting objects: 100% (536/536), done.
remote: Compressing objects: 100% (404/404), done.
remote: Total 1341 (delta 223), reused 395 (delta 129), pack-reused 805 (from 1)
Receiving objects: 100% (1341/1341), 2.59 GiB | 35.08 MiB/s, done.
Resolving deltas: 100% (405/405), done.
Updating files: 100% (605/605), done.
/content/PPI-Inhibitors
/content/PPI-Inhibitors/Data
/content/PPI-Inhibitors

✓ Repository cloned and datasets downloaded


In [4]:
# Mount Google Drive and download pre-computed features
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    %cd '/content/drive/MyDrive'
    !mkdir -p GNN-PPI-Inhibitor

    print("Downloading pre-computed protein features (this may take a few minutes)...")
    !gdown --id 1goeDiPZSKT1Xx3j00eNG9xlqYkLLv1gW -O GNN-PPI-Inhibitor/ProteinData_dict.pickle
    !gdown --id 1GOYEKLQCoGea9QQ72kujy0rdJKbUSYAE -O GNN-PPI-Inhibitor/DBD5_ProteinData_dict.pickle

    print("\n✓ Features downloaded")
else:
    print("⚠️  Running locally - make sure pre-computed features are available")

Mounted at /content/drive
/content/drive/MyDrive
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1goeDiPZSKT1Xx3j00eNG9xlqYkLLv1gW
To: /content/drive/MyDrive/GNN-PPI-Inhibitor/ProteinData_dict.pickle
100% 34.6M/34.6M [00:00<00:00, 35.2MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1GOYEKLQCoGea9QQ72kujy0rdJKbUSYAE
From (redirected): https://drive.google.com/uc?id=1GOYEKLQCoGea9QQ72kujy0rdJKbUSYAE&confirm=t&uuid=ea965873-28b7-4e50-b32a-1439f88c132c
To: /content/drive/MyDrive/GNN-PPI-Inhibitor/DBD5_ProteinData_dict.pickle
100% 3

---
## 2. Imports and CUDA Utilities

In [23]:
# Core imports
import warnings
from Bio.PDB import *
from Bio.PDB.NeighborSearch import NeighborSearch
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.utils.data.sampler import WeightedRandomSampler
from torch.autograd import Variable
import pickle
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from scipy import spatial
from Bio import SeqIO
from Bio.SeqIO import FastaIO
from itertools import product
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.preprocessing import normalize
import math
from Bio.Data import IUPACData
from Bio.PDB.Polypeptide import *

# CUDA setup
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")

def cuda(v):
    """Move tensor to CUDA if available"""
    if USE_CUDA:
        return v.cuda()
    return v

def toTensor(v, dtype=torch.float, requires_grad=False):
    """Convert to tensor and move to device"""
    return cuda(Variable(torch.tensor(v)).type(dtype).requires_grad_(requires_grad))

def toNumpy(v):
    """Convert tensor to numpy"""
    if USE_CUDA:
        return v.detach().cpu().numpy()
    return v.detach().numpy()

print(f"Device: {device}")
if USE_CUDA:
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  No GPU detected. Training will be very slow!")

Device: cuda:0
GPU: Tesla T4


---
## 3. Feature Extraction Functions

### CORRECTED implementations from repository:
- `atom1()`: 13-dim encoding
- `res1()`: 21-dim encoding
- `neigh1()`: **THE KEY** - heterogeneous neighbor lists

In [6]:
def atom1(structure):
    """
    One-hot encode atoms - 13 dimensions.

    Categories: ['C', 'CA', 'CB', 'CG', 'CH2', 'N', 'NH2', 'OG', 'OH', 'O1', 'O2', 'SE', '1']
    '1' = unknown atom type
    """
    atomslist = np.array(sorted(np.array(['C', 'CA', 'CB', 'CG', 'CH2', 'N','NH2',  'OG','OH', 'O1', 'O2', 'SE','1']))).reshape(-1,1)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(atomslist)

    atom_list = []
    for atom in structure.get_atoms():
        if atom.get_name() in atomslist:
            atom_list.append(atom.get_name())
        else:
            atom_list.append("1")

    atoms_onehot = enc.transform(np.array(atom_list).reshape(-1,1)).toarray()
    return atoms_onehot

print("✓ atom1() defined (13-dim)")

✓ atom1() defined (13-dim)


In [7]:
def res1(structure):
    """
    One-hot encode residues - 21 dimensions.

    20 amino acids + '1' for unknown
    """
    residuelist = np.array(sorted(np.array(['ALA', 'ARG', 'ASN', 'ASP', 'GLN', 'GLU', 'GLY', 'ILE', 'LEU', 'LYS',
                                            'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'CYS', 'HIS','1']))).reshape(-1,1)
    encr = OneHotEncoder(handle_unknown='ignore')
    encr.fit(residuelist)

    residue_list = []
    for atom in structure.get_atoms():
        if atom.get_parent().get_resname() in residuelist:
            residue_list.append((atom.get_parent()).get_resname())
        else:
            residue_list.append("1")

    res_onehot = encr.transform(np.array(residue_list).reshape(-1,1)).toarray()
    return res_onehot

print("✓ res1() defined (21-dim)")

✓ res1() defined (21-dim)


In [8]:
def neigh1(structure):
    """
    Calculate heterogeneous neighbors - THE KEY INNOVATION!

    Returns TWO separate neighbor lists:
    - neigh_same_res: 10 neighbors within SAME residue
    - neigh_diff_res: 10 neighbors in DIFFERENT residues

    This is what makes the GNN "heterogeneous".
    """
    atom_list = np.array([atom for atom in structure.get_atoms()])

    # Find neighbors within 6 Angstroms
    p4 = NeighborSearch(atom_list)
    neighbour_list = p4.search_all(6, level="A")
    neighbour_list = np.array(neighbour_list)

    # Sort by distance
    dist = np.array([np.linalg.norm(neighbour_list[i,0].get_coord() - neighbour_list[i,1].get_coord())
                     for i in range(len(neighbour_list))])
    place = np.argsort(dist)
    sorted_neighbour_list = neighbour_list[place]

    # Get atom and residue IDs
    source_vertex_list_atom_object = np.array(sorted_neighbour_list[:,0])
    len_source_vertex = len(source_vertex_list_atom_object)
    neighbour_vertex_with_respect_each_source_atom_object = np.array(sorted_neighbour_list[:,1])

    old_atom_number = []
    old_residue_number = []
    for i in atom_list:
        old_atom_number.append(i.get_serial_number())
        old_residue_number.append(i.get_parent().get_id()[1])

    old_atom_number = np.array(old_atom_number)
    old_residue_number = np.array(old_residue_number)

    total_atoms = len(atom_list)

    # Initialize neighbor arrays (-1 = no neighbor)
    neigh_same_res = np.array([[-1]*10 for i in range(total_atoms)])
    neigh_diff_res = np.array([[-1]*10 for i in range(total_atoms)])
    same_flag = [0]*total_atoms
    diff_flag = [0]*total_atoms

    # Fill neighbor arrays
    for i in range(len_source_vertex):
        source_atom_id = source_vertex_list_atom_object[i].get_serial_number()
        neigh_atom_id = neighbour_vertex_with_respect_each_source_atom_object[i].get_serial_number()
        source_atom_res = source_vertex_list_atom_object[i].get_parent().get_id()[1]
        neigh_atom_res = neighbour_vertex_with_respect_each_source_atom_object[i].get_parent().get_id()[1]

        # Find indices
        temp_index1 = np.where(source_atom_id == old_atom_number)[0]
        temp_index2 = np.where(neigh_atom_id == old_atom_number)[0]

        for i1 in temp_index1:
            if old_residue_number[i1] == source_atom_res:
                source_index = i1
                break
        for i1 in temp_index2:
            if old_residue_number[i1] == neigh_atom_res:
                neigh_index = i1
                break

        # Same residue neighbors
        if source_atom_res == neigh_atom_res:
            if int(same_flag[source_index]) < 10:
                neigh_same_res[source_index][same_flag[source_index]] = neigh_index
                same_flag[source_index] += 1

            if int(same_flag[neigh_index]) < 10:
                neigh_same_res[neigh_index][same_flag[neigh_index]] = source_index
                same_flag[neigh_index] += 1

        # Different residue neighbors
        elif source_atom_res != neigh_atom_res:
            if int(diff_flag[source_index]) < 10:
                neigh_diff_res[source_index][diff_flag[source_index]] = neigh_index
                diff_flag[source_index] += 1

            if int(diff_flag[neigh_index]) < 10:
                neigh_diff_res[neigh_index][diff_flag[neigh_index]] = source_index
                diff_flag[neigh_index] += 1

    return neigh_same_res, neigh_diff_res

print("✓ neigh1() defined - Creates heterogeneous neighbor lists")
print("  This is THE KEY innovation of the paper!")

✓ neigh1() defined - Creates heterogeneous neighbor lists
  This is THE KEY innovation of the paper!


### Utility Functions

In [9]:
def readFile(filename):
    """
    Read inhibitor data file.
    Format: name pdbid complex ligandid smiles label
    """
    with open(filename) as f:
        D = f.readlines()

    Name, PdbId, Ligandnames, SMILES, labels = [], [], [], [], []

    for d in tqdm(D, desc="Reading file"):
        if len(d.split()) == 6:
            name, inhibtedc, Pdbid, Ligandid, smiles, y = d.split()
            Name.append(name)
            PdbId.append(Pdbid)
            Ligandnames.append(Ligandid)
            SMILES.append(smiles)
            labels.append(float(y))

    return PdbId, Ligandnames, SMILES, labels


def processProtein(UniqueProtein, PdBloc):
    """
    Process PDB files to graph representation.
    Only needed if processing NEW proteins (not using pre-computed features).
    """
    PData_dict = {}

    for i in range(len(UniqueProtein)):
        print(f'Converting PDB to Graph: {i+1}/{len(UniqueProtein)}')
        UniqueProtein[i] = UniqueProtein[i].split('.pdb')[0]
        P1 = PdBloc + UniqueProtein[i] + '.pdb'

        parser = PDBParser()
        with warnings.catch_warnings(record=True):
            structure = parser.get_structure("", P1)

        one_hot_atom = atom1(structure)
        one_hot_res = res1(structure)
        neigh_same_res, neigh_diff_res = neigh1(structure)

        one_hot_atom = torch.tensor(one_hot_atom, dtype=torch.float32).to(device)
        one_hot_res = torch.tensor(one_hot_res, dtype=torch.float32).to(device)
        neigh_same_res = torch.tensor(neigh_same_res).to(device).long()
        neigh_diff_res = torch.tensor(neigh_diff_res).to(device).long()

        GNNData = [one_hot_atom, one_hot_res, neigh_same_res, neigh_diff_res]
        PData_dict[UniqueProtein[i]] = GNNData

    return PData_dict

print("✓ Utility functions defined")

✓ Utility functions defined


---
## 4. Model Architecture

All classes from repository included.

In [10]:
class GNN_First_Layer(nn.Module):
    """
    First GNN layer - processes atom and residue features.
    Weight matrices: Wv (atoms), Wr (residues), Wsr (same-res), Wdr (diff-res)
    """
    def __init__(self, filters, trainable=True, **kwargs):
        super(GNN_First_Layer, self).__init__()
        self.filters = filters
        self.trainable = trainable
        self.cuda_device = device

        self.Wv = nn.Parameter(torch.randn(13, self.filters, device=self.cuda_device, requires_grad=True))
        self.Wr = nn.Parameter(torch.randn(21, self.filters, device=self.cuda_device, requires_grad=True))
        self.Wsr = nn.Parameter(torch.randn(13, self.filters, device=self.cuda_device, requires_grad=True))
        self.Wdr = nn.Parameter(torch.randn(13, self.filters, device=self.cuda_device, requires_grad=True))
        self.neighbours = 10

    def forward(self, x):
        atoms, residues, same_neigh, diff_neigh = x

        node_signals = atoms @ self.Wv
        residue_signals = residues @ self.Wr
        neigh_signals_same = atoms @ self.Wsr
        neigh_signals_diff = atoms @ self.Wdr

        unsqueezed_same_neigh_indicator = (same_neigh > -1).unsqueeze(2)
        unsqueezed_diff_neigh_indicator = (diff_neigh > -1).unsqueeze(2)

        same_neigh_features = neigh_signals_same[same_neigh] * unsqueezed_same_neigh_indicator
        diff_neigh_features = neigh_signals_diff[diff_neigh] * unsqueezed_diff_neigh_indicator

        same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1, 1).unsqueeze(1).type(torch.float)

        same_norm[same_norm == 0] = 1
        diff_norm[diff_norm == 0] = 1

        neigh_same_atoms_signal = torch.sum(same_neigh_features, axis=1) / same_norm
        neigh_diff_atoms_signal = torch.sum(diff_neigh_features, axis=1) / diff_norm

        final_res = torch.relu(node_signals + residue_signals + neigh_same_atoms_signal + neigh_diff_atoms_signal)

        return final_res, same_neigh, diff_neigh

print("✓ GNN_First_Layer defined")

✓ GNN_First_Layer defined


In [11]:
class GNN_Layer(nn.Module):
    """
    Subsequent GNN layers.
    Weight matrices: Wsv (center), Wsr (same-res), Wdr (diff-res)
    """
    def __init__(self, filters, v_feats, trainable=True, **kwargs):
        super(GNN_Layer, self).__init__()
        self.v_feats = v_feats
        self.filters = filters
        self.trainable = trainable
        self.cuda_device = device

        self.Wsv = nn.Parameter(torch.randn(self.v_feats, self.filters, device=self.cuda_device, requires_grad=True))
        self.Wdr = nn.Parameter(torch.randn(self.v_feats, self.filters, device=self.cuda_device, requires_grad=True))
        self.Wsr = nn.Parameter(torch.randn(self.v_feats, self.filters, device=self.cuda_device, requires_grad=True))
        self.neighbours = 10

    def forward(self, x):
        Z, same_neigh, diff_neigh = x

        node_signals = Z @ self.Wsv
        neigh_signals_same = Z @ self.Wsr
        neigh_signals_diff = Z @ self.Wdr

        unsqueezed_same_neigh_indicator = (same_neigh > -1).unsqueeze(2)
        unsqueezed_diff_neigh_indicator = (diff_neigh > -1).unsqueeze(2)

        same_neigh_features = neigh_signals_same[same_neigh] * unsqueezed_same_neigh_indicator
        diff_neigh_features = neigh_signals_diff[diff_neigh] * unsqueezed_diff_neigh_indicator

        same_norm = torch.sum(same_neigh > -1, 1).unsqueeze(1).type(torch.float)
        diff_norm = torch.sum(diff_neigh > -1, 1).unsqueeze(1).type(torch.float)

        same_norm[same_norm == 0] = 1
        diff_norm[diff_norm == 0] = 1

        neigh_same_atoms_signal = torch.sum(same_neigh_features, axis=1) / same_norm
        neigh_diff_atoms_signal = torch.sum(diff_neigh_features, axis=1) / diff_norm

        final_res = torch.relu(node_signals + neigh_same_atoms_signal + neigh_diff_atoms_signal)

        return final_res, same_neigh, diff_neigh

print("✓ GNN_Layer defined")

✓ GNN_Layer defined


In [12]:
class Dense(nn.Module):
    """
    Dense layer with sigmoid activation.
    (Defined in repository but not used in final model - included for completeness)
    """
    def __init__(self, in_dims, out_dims, trainable=True, **kwargs):
        super(Dense, self).__init__()
        self.in_dims = in_dims
        self.out_dims = out_dims
        self.cuda_device = device

        self.W = nn.Parameter(torch.randn(self.in_dims, self.out_dims,
                                         device=self.cuda_device, requires_grad=True))

    def forward(self, x):
        Z = torch.sigmoid(torch.matmul(x, self.W))
        return Z

print("✓ Dense class defined")

✓ Dense class defined


In [13]:
class GNN(torch.nn.Module):
    """
    Complete 3-layer GNN.
    Architecture: 13+21 → 512 → 1024 → 512 → global mean pool
    """
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GNN_First_Layer(filters=512)
        self.conv2 = GNN_Layer(v_feats=512, filters=1024)
        self.conv3 = GNN_Layer(v_feats=1024, filters=512)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)

        # Global mean pooling
        x = x3[0]
        x = torch.sum(x, axis=0).view(1, -1)
        x = F.normalize(x)

        return x

print("✓ GNN model defined")

✓ GNN model defined


In [14]:
class IPPI_MLP_Net(nn.Module):
    """
    MLP for final inhibition prediction.
    Input: 2840 = 512 (GNN) + 280 (interface+seq) + 2048 (compound)
    """
    def __init__(self):
        super(IPPI_MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2840, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 100)
        self.fc6 = nn.Linear(100, 1)

    def forward(self, PFeatures, LigandFeatures, ProteinInterfaceF):
        P_all_Features = torch.hstack((PFeatures, ProteinInterfaceF))
        PC_Features = torch.hstack((P_all_Features, LigandFeatures))

        x = torch.tanh(self.fc1(PC_Features))
        x = torch.tanh(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc6(x)

        return x

print("✓ IPPI_MLP_Net defined")

✓ IPPI_MLP_Net defined


---
## 5. Dataset & Sampling Classes

All sampling methods from repository.

In [15]:
class CustomDataset(Dataset):
    """Simple dataset wrapper"""
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

print("✓ CustomDataset defined")

✓ CustomDataset defined


In [16]:
class BinaryBalancedSampler(Sampler):
    """
    Returns batches with 50% positive and 50% negative examples.
    This is the main sampler used in training.
    """
    def __init__(self, class_vector, batch_size=10):
        self.batch_size = batch_size
        self.class_vector = class_vector

        YY = np.array(self.class_vector)
        U, C = np.unique(YY, return_counts=True)
        M = U[np.argmax(C)]  # Majority class

        Midx = np.nonzero(YY == M)[0]  # Majority indices
        midx = np.nonzero(YY != M)[0]  # Minority indices

        # Oversample minority to match majority
        midx_ = np.random.choice(midx, size=len(Midx))

        self.YY = np.array(list(YY[Midx]) + list(YY[midx_]))
        self.idx = np.array(list(Midx) + list(midx_))

        self.n_splits = int(np.ceil(len(self.idx) / self.batch_size))
        self.equivalent_epochs = len(self.idx) / len(self.class_vector)

        print(f'Equivalent epochs in one iteration: {self.equivalent_epochs:.2f}')

    def gen_sample_array(self):
        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True)
        for tridx, ttidx in skf.split(self.idx, self.YY):
            yield np.array(self.idx[ttidx])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return self.n_splits

print("✓ BinaryBalancedSampler defined")

✓ BinaryBalancedSampler defined


In [17]:
class BalancedDataset(Dataset):
    """
    Alternative balanced dataset using sample weights.
    (BinaryBalancedSampler is used in main training - this is an alternative)
    """
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

        class_counts = np.bincount(self.labels)
        weights = 1. / torch.tensor(class_counts, dtype=torch.float)
        self.sample_weights = weights[labels]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


def create_balanced_loader(data, labels, batch_size=32):
    """
    Creates DataLoader with balanced batches using WeightedRandomSampler.
    Alternative to BinaryBalancedSampler.
    """
    dataset = BalancedDataset(data, labels)

    sampler = WeightedRandomSampler(
        weights=dataset.sample_weights,
        num_samples=len(dataset.sample_weights),
        replacement=True
    )

    loader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    return loader

print("✓ BalancedDataset & create_balanced_loader defined (alternatives)")

✓ BalancedDataset & create_balanced_loader defined (alternatives)


---
## 6. Load Pre-computed Features

In [18]:
# Set paths
if IN_COLAB:
    path = '/content/drive/MyDrive/GNN-PPI-Inhibitor/'
    githubpath = '/content/PPI-Inhibitors/'
else:
    path = 'GNN-PPI-Inhibitor/'
    githubpath = 'PPI-Inhibitors/'

print(f"Data path: {path}")
print(f"GitHub path: {githubpath}")

Data path: /content/drive/MyDrive/GNN-PPI-Inhibitor/
GitHub path: /content/PPI-Inhibitors/


In [19]:
# Load interface and sequence features (280-dim)
print("Loading interface/sequence features...")

Ubench5InterfaceandSeq_dict = pickle.load(open(githubpath + 'Features/NewUbench5InterfaceandSeq_dict.npy', "rb"))
Pos_seqandInterfaceF_dict = pickle.load(open(githubpath + 'Features/Pos_seqandInterfaceF_dict.npy', "rb"))

Complex_AllFeatures_dict = dict(list(Pos_seqandInterfaceF_dict.items()) + list(Ubench5InterfaceandSeq_dict.items()))

ComplexInterfaceFeatures = {}
for key in Complex_AllFeatures_dict:
    if len(key.split('_')) > 1:
        compname = key.split('_')[0]
        ComplexInterfaceFeatures[compname] = Complex_AllFeatures_dict[key]
    else:
        ComplexInterfaceFeatures[key] = Complex_AllFeatures_dict[key]

print(f"✓ Loaded features for {len(ComplexInterfaceFeatures)} complexes")

# Load compound fingerprints (2048-dim)
CompoundFingerprintFeaturesDict = pickle.load(open(githubpath + 'Features/Compound_Fingerprint_Features_Dict.npy', "rb"))
print(f"✓ Loaded fingerprints for {len(CompoundFingerprintFeaturesDict)} compounds")

Loading interface/sequence features...
✓ Loaded features for 290 complexes
✓ Loaded fingerprints for 8868 compounds


In [20]:
# Load protein GNN data
print("Loading protein structures...")

ProteinDataGNN_dict = pickle.load(open(path + 'ProteinData_dict.pickle', "rb"))
DBD5_ProteinDataGNN_dict = pickle.load(open(path + 'DBD5_ProteinData_dict.pickle', "rb"))

All_ProteinData_dict = dict(list(ProteinDataGNN_dict.items()) + list(DBD5_ProteinDataGNN_dict.items()))

# Move to CUDA
for d in All_ProteinData_dict:
    data = All_ProteinData_dict[d]
    All_ProteinData_dict[d] = [data[0].cuda(), data[1].cuda(), data[2].cuda(), data[3].cuda()]

print(f"✓ Loaded {len(All_ProteinData_dict)} protein structures")

Loading protein structures...
✓ Loaded 290 protein structures


---## 5B. External Validation Functions**FROM REPOSITORY CELL 12 - EXACT CODE**These functions enable testing on external datasets:- Recent Publications dataset- COVID-19 ACE2 inhibitors dataset

In [24]:
def getFP(s,r = 3,nBits =2048):
    compound = Chem.MolFromSmiles(s.strip())
    if compound is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(compound, r, nBits = nBits)
        #fp = pat.GetAvalonCountFP(compound,nBits=nBits)
        m = np.zeros((0, ), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fp, m)
        return m

def twomerFromSeq(s):
    k=2
    groups={'A':'1','V':'1','G':'1','I':'2','L':'2','F':'2','P':'2','Y':'3',
            'M':'3','T':'3','S':'3','H':'4','N':'4','Q':'4','W':'4',
            'R':'5','K':'5','D':'6','E':'6','C':'7'}
    crossproduct=[''.join (i) for i in product("1234567",repeat=k)]
    for i in range (0,len(crossproduct)): crossproduct[i]=int(crossproduct[i])
    ind=[]
    for i in range (0,len(crossproduct)): ind.append(i)
    combinations=dict(zip(crossproduct,ind))

    V=np.zeros(int((math.pow(7,k))))      #defines a vector of 343 length with zero entries
    try:
        for j in range (0,len(s)-k+1):
            kmer=s[j:j+k]
            c=''
            for l in range(0,k):
                c+=groups[kmer[l]]
                V[combinations[int(c)]]+=1
    except:
        count={'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0}
        for q in range(0,len(s)):
            if s[q]=='A' or s[q]=='V' or s[q]=='G':
                count['1']+=1
            if s[q]=='I' or s[q]=='L'or s[q]=='F' or s[q]=='P':
                count['2']+=1
            if s[q]=='Y' or s[q]=='M'or s[q]=='T' or s[q]=='S':
                count['3']+=1
            if s[q]=='H' or s[q]=='N'or s[q]=='Q' or s[q]=='W':
                count['4']+=1
            if s[q]=='R' or s[q]=='K':
                count['5']+=1
            if s[q]=='D' or s[q]=='E':
                count['6']+=1
            if s[q]=='C':
                count['7']+=1
        val=list(count.values()  )           #[ 0,0,0,0,0,0,0]
        key=list(count.keys()     )           #['1', '2', '3', '4', '5', '6', '7']
        m=0
        ind=0
        for t in range(0,len(val)):     #find maximum value from val
            if m<val[t]:
                m=val[t]
                ind=t
        m=key [ind]                     # m=group number of maximum occuring group alphabets in protein
        for j in range (0,len(s)-k+1):
            kmer=s[j:j+k]
            c=''
            for l in range(0,k):
                if kmer[l] not in groups:
                    c+=m
                else:
                    c+=groups[kmer[l]]
            V[combinations[int(c)]]+=1

    V=V/(len(s)-1)
    return np.array(V)


In [25]:
def chainLabel(Cname_T,xl_T,Cname,xl):
    """
    Cname_T: Target chain Name
    xl_T: Target chain co-ordinates
    Cname: Off Target chain Name
    xl: Off Target chain co-ordinates
    """
    tc = getCoords(xl_T)
    nc = getCoords(xl)
    D = getDist(tc, nc, thr = 8.0)
    feats=extract_feats(generate_pair_features(D,xl_T,xl))
    return feats

def generate_pair_features(dist_info,xl,xr):
    prot_dic=make_dic()
#    pdb.set_trace()
    for rec in dist_info:

        try:
            l_letter= three_to_one(xl[rec[0]].get_resname())
            r_letter= three_to_one(xr[rec[1]].get_resname())
#            print(l_letter,l_letter)
            if (l_letter,r_letter) in prot_dic.keys():
                prot_dic[(l_letter,r_letter)]+=1
            elif (r_letter,l_letter) in prot_dic.keys():
                prot_dic[(r_letter,l_letter)]+=1
        except:
            prot_dic[('_','_')]+=1
    return prot_dic

def getCoords(R):
    """
    Get atom coordinates given a list of biopython residues
    """
    Coords = []
    for (idx, r) in enumerate(R):
        v = [ak.get_coord() for ak in r.get_list()]
        Coords.append(v)
    return Coords

def InterfaceFeatures(Complexs,pdbloc):
    Found =  listdir(pdbloc)
    InterfaceFeatures=[];InterfaceFeatures=dict(InterfaceFeatures)
    comp_id=list(set(Complexs))
    for ids in range(len(comp_id)):
        if comp_id[ids]+'.pdb' in Found:
            stx=pdbloc+'/'+comp_id[ids]+'.pdb'#'/2XA0.pdb'
            chains=Struct2chain(stx)
            for j in range(len(chains)):
                Cname_T,seq_T,L_T,xl_T=chains[j]
                for k in range(j,len(chains)):
                    Cname,seq,L,xl=chains[k]
                    #if Cname_T!=Cname and Cname!=' 'and Cname_T!=' ':
                    name=comp_id[ids]#+'_'+Cname_T+'_2_'+Cname
                    Interface=chainLabel(Cname_T,xl_T,Cname,xl)
                    InterfaceF=np.array(Interface)
                    InterfaceF=normalize(np.atleast_2d(InterfaceF), norm='l2', copy=True, axis=1, return_norm=False)
                    if name not in InterfaceFeatures.keys():
                        InterfaceFeatures[name]=Interface
    #pickle.dump(InterfaceFeatures, open(path+Filename+"_InterfaceFeatures.npy", "wb"))
    return InterfaceFeatures

def getDist(C0, C1, thr=np.inf):
    """

    """
    N0 = []
    N1 = []
    for i in range(len(C0)):
        for j in range(len(C1)):
            d = spatial.distance.cdist(C0[i], C1[j]).min()
            # dji=spatial.distance.cdist(C1[j], C0[i]).min()
            #d=min(dij,dji)
            #print d
            if (d < thr):  # and not np.isnan(self.Phi[i]) and not np.isnan(self.Phi[j])
                N0.append((i, j, d))
                N1.append((j, i, d))
    return (N0, N1)

In [26]:
def prot_feats_seq(seq):
    #Interfacedict=pickle.load(open(path+"InhibitorNewModel2022/InterfaceFeatures2chainsSVM.npy","rb"))
    #InterfaceF=Interfacedict[complexname]
    aa=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    f=[]
    X = ProteinAnalysis(str(seq))
    X.molecular_weight() #throws an error if 'X' in sequence. we skip such sequences
    p=X.get_amino_acids_percent()
    dp=[]
    for a in aa:
        dp.append(p[a])
    dp=np.array(dp)
    dp=normalize(np.atleast_2d(dp), norm='l2', copy=True, axis=1, return_norm=False)
    f.extend(dp[0])

    tm=np.array(twomerFromSeq(str(seq)))
    tm=normalize(np.atleast_2d(tm), norm='l2', copy=True, axis=1,return_norm=False)
    f.extend(tm[0])
    return np.array(f)

def Struct2chain(stx):
    """
    Seq: sequence of the chain
    seq_L:sequence Length
    """
    p = PDBParser()
    L=[]
    stx=p.get_structure('X',stx)
    for model in stx:
        for C in model:
            RL=[]
            for R in C:
                RL.append(R)
            pp=PPBuilder().build_peptides(C)
            if len(pp)==0:
                pp=CaPPBuilder().build_peptides(C)
            seq=''.join([str(p.get_sequence()) for p in pp])
            #seq=''.join([p.get_sequence().tostring() for p in pp])
            seq_L=len(seq)
            L.append((C.full_id[2],seq,seq_L,RL))
    return L

def extract_feats(dic):
    feats=[]
    key_list=np.load('/content/PPI-Inhibitors/Features/'+'prote_letter_pair_keys.npy')#to keep features order same
    for key in key_list:
#        pdb.set_trace()
        feats.append(dic[(key[0].decode('utf-8'),key[1].decode('utf-8'))])

    return feats

def LoadProtein_SVM_Features(UniqueProtein,Pdbloc):
    pdbname=listdir(Pdbloc)
    InterfaceFeatures=[];InterfaceFeatures=dict(InterfaceFeatures)
    SequenceFeatures=[];SequenceFeatures=dict(SequenceFeatures)
    AllFeatures=[];AllFeatures=dict(AllFeatures)
    for  b in range(len(UniqueProtein)):
        if UniqueProtein[b]+'.pdb'in pdbname:
            stx=Pdbloc+UniqueProtein[b]+'.pdb'#directory+'/2XA0.pdb'#
            chains=Struct2chain(stx)
            #########Interface Features
            for j in range(len(chains)):
                Cname_T,seq_T,L_T,xl_T=chains[j]
                for k in range(j,len(chains)):
                    Cname,seq,L,xl=chains[k]
                    #if Cname_T!=Cname and Cname!=' 'and Cname_T!=' ':
                    name=UniqueProtein[b]#+'_'+Cname_T+'_2_'+Cname
                    Interface=chainLabel(Cname_T,xl_T,Cname,xl)
                    seq_TF=prot_feats_seq(seq_T)
                    seq_NTF=prot_feats_seq(seq)
                    SeQFeatures=(seq_TF+seq_NTF)/2
                    InterfaceF=np.array(Interface)
                    InterfaceF=normalize(np.atleast_2d(InterfaceF), norm='l2', copy=True, axis=1, return_norm=False)
                    if name not in InterfaceFeatures.keys():
                        InterfaceFeatures[name]=Interface
                        SequenceFeatures[name]=SeQFeatures
                        AllFeatures[name]=np.append(SeQFeatures,Interface)
    return InterfaceFeatures,SequenceFeatures,AllFeatures

In [27]:
def External_GenerateRandomNegative(posexamples):
    NegtiveRatio=1
    ###SuperDrugbank###Names
    SuperdrugNames=pd.read_excel('/content/PPI-Inhibitors/Data/approved_drugs_chemical_structure_identifiers.xlsx',usecols="B").values#'approved_drugs_chemical_structure_identifiers.xlsx'
    SuperdrugNames=SuperdrugNames[1:]
    SuperdrugNames = np.array([s[0] for s in SuperdrugNames])
    ###############SuperDrugbank
    df_Superdrug=pd.read_excel('/content/PPI-Inhibitors/Data/approved_drugs_chemical_structure_identifiers.xlsx',usecols="C").values
    df_Superdrug=df_Superdrug[1:]
    ###
    df_Superdrug_Compounds=np.array([c[0] for c in df_Superdrug])#3638
    SuperDrug_dict=dict (zip (SuperdrugNames,df_Superdrug_Compounds))
    ################
    #path='/content/drive/MyDrive/GNN-PPI-Inhibitor/'
    DBD5_ProteinData_dict=pickle.load(open('/content/PPI-Inhibitors/Features/NewUbench5InterfaceandSeq_dict.npy',"rb"))
    Ubench5CompNames=list (set (list (DBD5_ProteinData_dict.keys())))
    ####
    AllNeg=[];AllPos=[];complex_ligand_dict={};
    for key,val in  posexamples:
      #print(key,val,posexamples[key,val][1])
      if key not in complex_ligand_dict:
        complex_ligand_dict[key]=posexamples[key,val][1]
      else:
        #print("else",key,val)
        complex_ligand_dict[key]=np.append( complex_ligand_dict.get(key, ()) ,posexamples[key,val][1])
    Complexnames=list (complex_ligand_dict.keys())
    totalcomp=list (set (complex_ligand_dict.keys()))
    for everycomp in totalcomp:
        origanlL=complex_ligand_dict[everycomp]
        #print(origanlL)
        #print("complexname=",everycomp,"origanlInhibitors",len(origanlL))
        pos=[(everycomp,origanlL[t]) for t in range(len(origanlL))]
        #print(pos)
        NN =NegtiveRatio*len(pos)
        negs = []
        AllPos.extend(pos)
        while (len(negs)<NN):# and len(negs)<(len(totalligands_train)-len(origanlL)):
            LigandR = random.choice(SuperdrugNames)
            LigandR_smile=SuperDrug_dict[LigandR]
            Npair=((everycomp,LigandR_smile))
            if LigandR  not in origanlL and Npair not in AllNeg and Npair not in AllPos and getFP(LigandR_smile) is not None:
                negs.append(Npair)
#                print("Npair SuperdrugNames",Npair)
        AllNeg.extend(negs)
    #print("N=",len(AllNeg),"P",len(AllPos))
    #print("second method Cr")
    for everycomp in totalcomp:
        origanlL=complex_ligand_dict[everycomp]
#        print("everycomp=",everycomp,"origanlL",len(origanlL))
        pos=[(everycomp,origanlL[t]) for t in range(len(origanlL))]
        NN =NegtiveRatio*len(pos)
        negs = []
        while (len(negs)<NN):
            for everyL in origanlL:
                ComplexR = random.choice(Ubench5CompNames)
                Npair=((ComplexR,everyL))
                if ComplexR!=everycomp and Npair not in AllNeg and Npair not in AllPos:
    #                    print("ComplexR,everycomp)",Npair)
                    negs.append(Npair)
        AllNeg.extend(negs)
        ###################
    #print("N=",len(AllNeg),"P",len(AllPos))
    return np.array(AllPos),np.array(AllNeg),SuperDrug_dict

In [28]:
def PredictScorefromFile(filename,Pdbloc,Pscaler,Cscaler,trainedModel_IPPI,train_GNN,LOCOcomplexname):
  githubpath='/content/PPI-Inhibitors/'
#filename,Pdbloc,Pscaler,Cscaler,trainedModel_IPPI,train_GNN=githubpath+'Data/External data/2dyh_all.txt',githubpath+'Data/External data/pdb/',Pscaler,Cscaler,IPPI_Net,GNN_model
  with open(filename) as f:
    D = f.readlines()
  InhibitedComp=[];PdbId=[];Ligandnames=[];SMILES=[];targets=[];
  All_data_list=[]
  from tqdm import tqdm as tqdm
  #2XA0_A_2_B 2O21 2XA0 43B c1ccc(cc1)CCc2nc3cc(ccc3s2)c4ccc(cc4)C(=O)NS(=O)(=O)c5ccc(c(c5)[N](=O)[O-])NCCSc6ccccc6  1
  for d in tqdm(D):
      Pdbid,inhibtedc,Ligandid,smiles = d.split()
      if getFP(smiles) is not None:
        PdbId.append(Pdbid);Ligandnames.append(Ligandid);SMILES.append(smiles);InhibitedComp.append(inhibtedc);#labels.append(float (y));
  ################
  """
  Result_dict={}
  pos=dict (zip(zip(PdbId, Ligandnames),zip(InhibitedComp,SMILES)))
  Pos,Negs,SuperDrug_dict=External_GenerateRandomNegative(pos)
  poslabel=1.0*np.ones(len(Pos));neglabel=-1.0*np.ones(len(Negs));targets=np.append(poslabel,neglabel )
  All_examples=[];All_examples.extend(Pos);All_examples.extend(Negs)
  #Write File for External
  External_All_Examples=open('/content/drive/MyDrive/GNN-PPI-Inhibitor/_'+LOCOcomplexname+'_'+filename.split('.txt')[0].split('/')[-1]+'_External_All_Examples.txt',"w")
  """
  complexnames=[];SMILES=[];targets=[];
  #/content/PPI-Inhibitors/Data/External data/2dyh_all_External_All_Examples.txt
  with open('/content/PPI-Inhibitors/Data/External data/'+filename.split('.txt')[0].split('/')[-1]+'_External_All_Examples.txt') as f:
    D = f.readlines()
  for d in tqdm(D):
      complexname,smiles,target= d.split()
      complexnames.append(complexname);SMILES.append(smiles);targets.append(target);#All_examples.append()
  pdbname=listdir(Pdbloc);mypdb=[]
  for p in pdbname:
    if p.split('.pdb')[0] in Pdbid:
      mypdb.append(p)
  UniqueProtein=list (set (mypdb))
  External_Protein_GNN_Data_dict=PrepairDataset.processProtein(UniqueProtein,Pdbloc)
  ##########for Seq+interface features
  #pdbname=listdir(Pdbloc)
  s,i,External_ProteinSeqandInterfaceData_dict=LoadProtein_SVM_Features(UniqueProtein,Pdbloc)
  ##############3 for sequence fedatures of DBD5 pdb's
  Ubench5InterfaceandSeq_dict=pickle.load(open(githubpath+'Features/NewUbench5InterfaceandSeq_dict.npy',"rb"))
  All_External_ProteinSeqandInterfaceData_dict=dict( list (External_ProteinSeqandInterfaceData_dict.items())+list (Ubench5InterfaceandSeq_dict.items()))
  #Testing
  DBD5_Protein_GNN_Data_dict=pickle.load(open(path+'DBD5_ProteinData_dict.pickle',"rb"))
  #Pos_ProteinData_dict=pickle.load(open(path+'ProteinData_dict.pickle',"rb"))
  All_Protein_GNN_Data_dict=dict( list (External_Protein_GNN_Data_dict.items())+list (DBD5_Protein_GNN_Data_dict.items()))
  for d in All_Protein_GNN_Data_dict:
    data=All_Protein_GNN_Data_dict[d]
    All_Protein_GNN_Data_dict[d]=[data[0].cuda(),data[1].cuda(),data[2].cuda(),data[3].cuda()]
  #####################
  Cttname=[];Ctt=[];Pttname=[];Ptt=[];
  for (complexname,ligandsmile) in zip(complexnames,SMILES):#All_examples:#
    Cttname.append(ligandsmile);Ctt.append(getFP(ligandsmile));
    Pttname.append(complexname);Ptt.append(All_External_ProteinSeqandInterfaceData_dict[complexname][0:69]);
  #standarization

  Ctt = Cscaler.transform(Ctt)
  Cttdict=dict (zip (Cttname,torch.FloatTensor( Ctt).cuda()))
  Ptt = Pscaler.transform(Ptt)
  Pttdict=dict (zip (Pttname,torch.FloatTensor( Ptt).cuda()))
  #########
  Y_t,Z,Targets=[],[],[]
  for target,(complexname,ligandsmile) in zip(targets,zip(complexnames,SMILES)):#zip(targets,All_examples):#
    #target=targets[nt]
    #IPPI_Net(All_ProteinData_dict[GNNcomplex],Cttdict[Ligandname],Pttdict[GNNcomplex],GNN_model)
    #External_All_Examples.write(complexname+' '+ligandsmile+' '+str(target)+'\n')
    test_score=trainedModel_IPPI(All_Protein_GNN_Data_dict[complexname],Cttdict[ligandsmile],Pttdict[complexname],train_GNN)
    #print (test_score)
    test_score=test_score.cpu().data.numpy()[0]
    Z.append(test_score);Targets.append(float (target))
    #Result_dict[(complexname,Ligandname)]=test_score
  return Z,Targets

In [29]:
# Load training examples
print("Loading training examples...")

with open(githubpath + 'Data/WriteAllexamplesRandomBindersIdsAll_24JAN_Binary.txt') as f:
    D = f.readlines()

Labels, Ligandnames, Complexs, TestPoscomplexes = [], [], [], []

for d in tqdm(D, desc="Parsing"):
    if len(d.split()) == 4:
        TestPoscomp, Complexname, Ligandname, label = d.split()
    else:
        TestPoscomp, Complexname, Ligandname, label = d.split()[0], d.split()[1], (' ').join(d.split()[2:-1]), d.split()[-1]

    TestPoscomplexes.append(TestPoscomp)
    Ligandnames.append(Ligandname)
    Complexs.append(Complexname)
    Labels.append(float(label))

Allexamples = dict(zip(zip(TestPoscomplexes, zip(Complexs, Ligandnames)), Labels))

print(f"\n✓ {len(Allexamples)} examples")
print(f"  Positive: {sum(Labels)}")
print(f"  Negative: {len(Labels) - sum(Labels)}")

# Load class ratios
classratio_dict = pickle.load(open(githubpath + 'Features/Classratio_GNNdict.npy', 'rb'))
print(f"✓ Class ratios loaded")

Loading training examples...


Parsing: 100%|██████████| 15695/15695 [00:00<00:00, 791344.82it/s]


✓ 11378 examples
  Positive: 857.0
  Negative: 14838.0
✓ Class ratios loaded


---
## 7. Leave-One-Complex-Out Cross-Validation

This will train 23 models (one per complex) and evaluate on held-out complex.

**Expected runtime:** 12-24 hours on GPU

In [36]:
# Prepare for cross-validation
Alldata = list(Allexamples.keys())
KK = [k[0].split('_')[0] for k in Alldata]
groups = pd.DataFrame(KK)
gkf = GroupKFold(n_splits=len(set(KK)))

Complexs = np.array(Complexs)
Ligandnames = np.array(Ligandnames)
Labels = np.array(Labels)
Alldata = np.array(Alldata, dtype=object)

AUC_ROC_final, Avg_P_final, Y_score, Y_t = [], [], [], []

test_complexes = ['3D9T', '1BKD', '4ESG', '2FLU', '1YCQ', '2XA0', '3TDU', '2B4J', '3DAB',
                  '3UVW', '2RNY', '4AJY', '1F47', '1YCR', '4QC3', '1NW9', '2E3K', '4YY6',
                  '4GQ6', '3WN7', '1BXL', '1Z92']


print(f"Running LOCO CV on {len(test_complexes)} complexes")
print("This will take 12-24 hours on GPU...\n")

Running LOCO CV on 22 complexes
This will take 12-24 hours on GPU...



In [37]:
# ========================================
# ⚡ FAST MODE - COMPLETE CONFIGURATION
# ========================================

from sklearn.model_selection import train_test_split

# 1. Reduce number of complexes
test_complexes = ['3D9T']

# 2. Training parameters
PATIENCE = 2
# bsize = 1024
bsize = 2048
# NUM_EPOCHS = 5
NUM_EPOCHS = 1
SAMPLE_FRACTION = 0.05  # 5% of data
Max_Per_Complex = 1000
# Create loaders

# 3. Dataset reduction function
def reduce_training_data(train, fraction=0.3, max_per_complex=Max_Per_Complex):
    """Reduce training data intelligently"""

    # Get labels
    train_labels = np.array([Allexamples[t[0], t[1]] for t in train])

    # Strategy 1: Global stratified sampling
    if len(train) * fraction > max_per_complex:
        target_size = int(len(train) * fraction)
    else:
        target_size = max_per_complex

    if target_size < len(train):
        train_reduced, _ = train_test_split(
            train,
            train_size=target_size,
            stratify=train_labels,
            random_state=42
        )
        return train_reduced

    return train

# 4. Apply to all data before loop starts
print("⚡ FAST MODE ACTIVATED")
print("="*60)
print(f"Complexes: {len(test_complexes)} (instead of 23)")
print(f"Epochs: {NUM_EPOCHS} (instead of 5)")
print(f"Batch size: {bsize}")
print(f"Dataset reduction: 30% stratified sampling")
print(f"Estimated time: 30-60 minutes")
print(f"Expected AUC-ROC: ~0.82-0.84")
print("="*60)

⚡ FAST MODE ACTIVATED
Complexes: 1 (instead of 23)
Epochs: 1 (instead of 5)
Batch size: 2048
Dataset reduction: 30% stratified sampling
Estimated time: 30-60 minutes
Expected AUC-ROC: ~0.82-0.84


In [38]:
# ============================================================================
# Initialize External Validation Accumulators
# ============================================================================

ExternalscoresLOCO = []
Externallabels = []
covid19_ExternalscoresLOCO = []
covid19_Externallabels = []

print("✓ External validation accumulators initialized")

✓ External validation accumulators initialized


---## 7B. External Validation Status**✅ ENABLED BY DEFAULT**External validation is now integrated into the training loop. For each complex:- Tests on Recent Publications dataset- Tests on COVID-19 ACE2 inhibitors dataset- Results accumulated and shown after training completes**To disable:** Comment out the external validation code block in the training loop.**Requirements:**- External data files from repository (auto-downloaded in setup)- Pre-computed features (included)- Adds ~5-10 min per complex (~2-4 hours total for all 23 complexes)

In [39]:
# Main training loop
for trainindex, testindex in gkf.split(KK, KK, groups=groups):
    train, test = Alldata[trainindex], Alldata[testindex]
    #####Reduce Training Set#########
    train = reduce_training_data(train, fraction=SAMPLE_FRACTION)
    ##################################
    test_complex_name = test[0][0].split('_')[0]

    if test_complex_name not in test_complexes:
        continue

    print(f"\n{'='*60}")
    print(f"Testing on: {test_complex_name}")
    print(f"{'='*60}")

    # Prepare data
    Ctr, Ptr, y_train, Ctrname, Ptrname = [], [], [], [], []
    for t in train:
        Ctrname.append(t[1][1])
        Ctr.append(CompoundFingerprintFeaturesDict[t[1][1]])
        GNNcomp = t[1][0].split('_')[0]
        Ptrname.append(GNNcomp)
        Ptr.append(ComplexInterfaceFeatures[GNNcomp])
        y_train.append(Allexamples[t[0], t[1]])

    Ctt, Ptt, y_test, Cttname, Pttname = [], [], [], [], []
    for t in test:
        GNNcomp = t[1][0].split('_')[0]
        Cttname.append(t[1][1])
        Ctt.append(CompoundFingerprintFeaturesDict[t[1][1]])
        Pttname.append(GNNcomp)
        Ptt.append(ComplexInterfaceFeatures[GNNcomp])
        y_test.append(Allexamples[t[0], t[1]])

    # Standardize
    Pscaler = StandardScaler().fit(Ptr)
    Cscaler = StandardScaler().fit(Ctr)
    Ctr = Cscaler.transform(Ctr)
    Ptr = Pscaler.transform(Ptr)
    Ptt = Pscaler.transform(Ptt)
    Ctt = Cscaler.transform(Ctt)

    Ptrdict = dict(zip(Ptrname, torch.FloatTensor(Ptr).cuda()))
    Ctrdict = dict(zip(Ctrname, torch.FloatTensor(Ctr).cuda()))
    Cttdict = dict(zip(Cttname, torch.FloatTensor(Ctt).cuda()))
    Pttdict = dict(zip(Pttname, torch.FloatTensor(Ptt).cuda()))

    y_train = np.array(y_train)

    print(f"Train: {len(y_train)} ({np.sum(y_train):.0f} pos)")
    print(f"Test: {len(y_test)} ({np.sum(y_test):.0f} pos)")

    # Initialize models
    IPPI_Net = IPPI_MLP_Net().cuda()
    GNN_model = GNN().cuda()

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(list(IPPI_Net.parameters()) + list(GNN_model.parameters()),
                          lr=0.001, weight_decay=0.0)


    dataset = CustomDataset(train[:,1], y_train.astype('int'))
    batch_sampler = BinaryBalancedSampler(y_train.astype('int'), bsize)
    loader = DataLoader(dataset, batch_sampler=batch_sampler)

    test_dataset = CustomDataset(test[:,1], np.array(y_test).astype('int'))
    test_loader = DataLoader(test_dataset, batch_size=bsize, shuffle=False)

    # Training
    Loss = []
    best_result = 0.0
    best_model = None

    for epoch in range(NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")

        for (batch_pids, batch_cids), batch_labels in tqdm(loader, desc="Training"):
            GNN_model.train()
            IPPI_Net.train()

            pids = [p.split('_')[0] for p in batch_pids]
            G_dict = {p: GNN_model(All_ProteinData_dict[p]) for p in set(pids)}
            GNN_features = torch.vstack([G_dict[p] for p in pids])
            del G_dict

            interface_features = torch.vstack([Ptrdict[p] for p in pids])
            compound_features = torch.vstack([Ctrdict[c] for c in batch_cids])

            output = IPPI_Net(GNN_features, compound_features, interface_features)

            V = np.min(list(classratio_dict.values()))
            weights = toTensor(np.array([classratio_dict[p]/V if batch_labels[i]==1 else 1.0
                                        for i,p in enumerate(pids)]))

            criterion = nn.BCEWithLogitsLoss(weight=None)
            loss = criterion(output.flatten(), batch_labels.float().cuda())
            Loss.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Validation
            GNN_model.eval()
            IPPI_Net.eval()
            Z, Y = [], []

            with torch.no_grad():
                for (batch_pids, batch_cids), batch_labels in test_loader:
                    pids = [p.split('_')[0] for p in batch_pids]
                    G_dict = {p: GNN_model(All_ProteinData_dict[p]) for p in set(pids)}
                    GNN_features = torch.vstack([G_dict[p] for p in pids])
                    del G_dict

                    interface_features = torch.vstack([Pttdict[p] for p in pids])
                    compound_features = torch.vstack([Cttdict[c] for c in batch_cids])

                    output = IPPI_Net(GNN_features, compound_features, interface_features)
                    Z.extend(output.cpu().flatten().numpy())
                    Y.extend(batch_labels.cpu().flatten().numpy())

                aucroc = roc_auc_score(np.array(Y), np.array(Z))

                if aucroc > best_result:
                    best_result = aucroc
                    best_model = (GNN_model.state_dict(), IPPI_Net.state_dict())

    # Evaluate best model
    IPPI_Net.load_state_dict(best_model[1])
    GNN_model.load_state_dict(best_model[0])

    GNN_model.eval()
    IPPI_Net.eval()
    Zb, Yb = [], []

    with torch.no_grad():
        for (batch_pids, batch_cids), batch_labels in test_loader:
            pids = [p.split('_')[0] for p in batch_pids]
            G_dict = {p: GNN_model(All_ProteinData_dict[p]) for p in set(pids)}
            GNN_features = torch.vstack([G_dict[p] for p in pids])
            del G_dict

            interface_features = torch.vstack([Pttdict[p] for p in pids])
            compound_features = torch.vstack([Cttdict[c] for c in batch_cids])

            output = IPPI_Net(GNN_features, compound_features, interface_features)
            Zb.extend(output.cpu().flatten().numpy())
            Yb.extend(batch_labels.cpu().flatten().numpy())

    aucrocb = roc_auc_score(np.array(Yb), np.array(Zb))
    aucprb = average_precision_score(Yb, Zb)

    print(f"\n{test_complex_name}: AUC-ROC={aucrocb:.4f}, AUC-PR={aucprb:.4f}")

    AUC_ROC_final.append(aucrocb)
    Avg_P_final.append(aucprb)
    Y_score.extend(Zb)
    Y_t.extend(Yb)

    torch.save(best_model[1], path + f'/IPPI_Net_{test_complex_name}')
    torch.save(best_model[0], path + f'/GNN_model_{test_complex_name}')

    # ========================================
    # EXTERNAL VALIDATION (Enabled by Default)
    # ========================================
    try:
        print(f"\nExternal validation for {test_complex_name}...")

        # Test on Recent Publications
        External_score, External_labels = PredictScorefromFile(
            githubpath + '/Data/External data/2dyh_all.txt',
            githubpath + '/Data/External data/pdb/',
            Pscaler, Cscaler, IPPI_Net, GNN_model, test_complex_name)

        ExternalscoresLOCO.extend(External_score)
        Externallabels.extend(External_labels)
        External_Auc = roc_auc_score(External_labels, External_score)
        External_AP = average_precision_score(External_labels, External_score)
        print(f"  Recent Pubs - AUC-ROC: {External_Auc:.3f}, AUC-PR: {External_AP:.3f}")

        # Test on COVID-19
        Covid19_External_score, Covid19_External_labels = PredictScorefromFile(
            githubpath + '/Data/External data/HansonACE2hits.txt',
            githubpath + '/Data/External data/pdb/',
            Pscaler, Cscaler, IPPI_Net, GNN_model, test_complex_name)

        covid19_Externallabels.extend(Covid19_External_labels)
        covid19_ExternalscoresLOCO.extend(Covid19_External_score)
        Covid19_External_Auc = roc_auc_score(Covid19_External_labels, Covid19_External_score)
        Covid19_External_AP = average_precision_score(Covid19_External_labels, Covid19_External_score)
        print(f"  COVID-19 - AUC-ROC: {Covid19_External_Auc:.3f}, AUC-PR: {Covid19_External_AP:.3f}")

    except Exception as e:
        print(f"  ⚠️  External validation failed: {e}")
        print(f"     Continuing with cross-validation results...")


print("\n" + "="*60)
print("CROSS-VALIDATION COMPLETE")
print("="*60)
print(f"\nMean AUC-ROC: {np.mean(AUC_ROC_final):.4f} ± {np.std(AUC_ROC_final):.4f}")
print(f"Mean AUC-PR: {np.mean(Avg_P_final):.4f} ± {np.std(Avg_P_final):.4f}")


Testing on: 3D9T
Train: 1000 (77 pos)
Test: 664 (28 pos)
Equivalent epochs in one iteration: 1.85

Epoch 1/1


Training:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

---
## 8. Visualization

In [ ]:
# Plot results
Y_score = np.array(Y_score)
Y_t = np.array(Y_t)

fpr, tpr, _ = roc_curve(Y_t, Y_score)
Auc = roc_auc_score(Y_t, Y_score)

precision, recall, _ = precision_recall_curve(Y_t, Y_score)
aucpr = average_precision_score(Y_t, Y_score)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# AUC-ROC
ax1.plot(fpr, tpr, color='k', marker='d', markersize=3, label=f'AUC: {Auc:.2f}')
ax1.plot([0, 1], [0, 1], 'k--', alpha=0.3)
ax1.set_xlabel('FPR', fontsize=12)
ax1.set_ylabel('TPR', fontsize=12)
ax1.set_title('AUC-ROC', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)
ax1.legend()

# AUC-PR
ax2.plot(recall, precision, color='m', marker=',', label=f'AUC-PR: {aucpr:.2f}')
baseline = np.sum(Y_t) / len(Y_t)
ax2.axhline(baseline, color='k', linestyle='--', alpha=0.3, label=f'Baseline: {baseline:.2f}')
ax2.set_xlabel('Recall', fontsize=12)
ax2.set_ylabel('Precision', fontsize=12)
ax2.set_title('AUC-PR', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.savefig('ppi_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Results plotted")

---## 8B. External Validation Results**OPTIONAL:** If external validation was run, these are the overall results.

In [ ]:
# ============================================================================
# EXTERNAL VALIDATION SUMMARY (Optional)
# ============================================================================

if len(ExternalscoresLOCO) > 0:
    print("\n" + "="*80)
    print("EXTERNAL VALIDATION - OVERALL RESULTS")
    print("="*80)

    # Recent Publications
    Overall_External_Auc = roc_auc_score(Externallabels, ExternalscoresLOCO)
    Overall_External_AP = average_precision_score(Externallabels, ExternalscoresLOCO)

    print(f"\nRecent Publications Dataset:")
    print(f"  Examples: {len(Externallabels)}")
    print(f"  AUC-ROC: {Overall_External_Auc:.3f}")
    print(f"  AUC-PR:  {Overall_External_AP:.3f}")
    print(f"  Expected: AUC-ROC ~0.82, AUC-PR ~0.45")

    # COVID-19
    if len(covid19_ExternalscoresLOCO) > 0:
        Overall_Covid19_Auc = roc_auc_score(covid19_Externallabels, covid19_ExternalscoresLOCO)
        Overall_Covid19_AP = average_precision_score(covid19_Externallabels, covid19_ExternalscoresLOCO)

        print(f"\nCOVID-19 ACE2 Inhibitors Dataset:")
        print(f"  Examples: {len(covid19_Externallabels)}")
        print(f"  AUC-ROC: {Overall_Covid19_Auc:.3f}")
        print(f"  AUC-PR:  {Overall_Covid19_AP:.3f}")
        print(f"  Expected: AUC-ROC ~0.78, AUC-PR ~0.42")

    # Plot external validation results
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Recent Publications
    fpr, tpr, _ = roc_curve(Externallabels, ExternalscoresLOCO)
    axes[0].plot(fpr, tpr, 'b-', linewidth=2,
                label=f'AUC-ROC: {Overall_External_Auc:.3f}')
    axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.3)
    axes[0].set_xlabel('False Positive Rate', fontsize=12)
    axes[0].set_ylabel('True Positive Rate', fontsize=12)
    axes[0].set_title('External: Recent Publications', fontsize=14, fontweight='bold')
    axes[0].grid(alpha=0.3)
    axes[0].legend()

    # COVID-19
    if len(covid19_ExternalscoresLOCO) > 0:
        fpr, tpr, _ = roc_curve(covid19_Externallabels, covid19_ExternalscoresLOCO)
        axes[1].plot(fpr, tpr, 'r-', linewidth=2,
                    label=f'AUC-ROC: {Overall_Covid19_Auc:.3f}')
        axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.3)
        axes[1].set_xlabel('False Positive Rate', fontsize=12)
        axes[1].set_ylabel('True Positive Rate', fontsize=12)
        axes[1].set_title('External: COVID-19 ACE2', fontsize=14, fontweight='bold')
        axes[1].grid(alpha=0.3)
        axes[1].legend()

    plt.tight_layout()
    plt.savefig('external_validation_results.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("\n✓ External validation complete!")
else:
    print("\n⚠️  External validation was not run")
    print("   (This is optional - cross-validation results are the main results)")

---## 9. Summary### Cross-Validation Results (Main):- **Expected AUC-ROC:** 0.86 ± 0.09- **Expected AUC-PR:** 0.39 ± 0.24### External Validation Results (Optional):- **Recent Pubs - Expected:** AUC-ROC ~0.82, AUC-PR ~0.45- **COVID-19 - Expected:** AUC-ROC ~0.78, AUC-PR ~0.42### Expected vs Actual Results:- **Expected AUC-ROC:** 0.86 ± 0.09- **Expected AUC-PR:** 0.39 ± 0.24### Key Components:- ✅ Corrected atom1() (13-dim)- ✅ Corrected res1() (21-dim)- ✅ Corrected neigh1() (heterogeneous)- ✅ Complete GNN architecture- ✅ All sampling classes- ✅ LOCO cross-validation### References:- **Paper:** https://doi.org/10.1101/2024.08.23.609286- **Repository:** https://github.com/adibayaseen/PPI-Inhibitors